# Renomear arquivos da pasta BGS

Este notebook renomeia todos os arquivos da pasta `bgs/` para um formato sequencial: `bg1.png`, `bg2.png`, etc.

In [ ]:
import os
import pathlib
from PIL import Image

# Pasta dos backgrounds
bgs_dir = pathlib.Path("bgs")

print(f"Verificando pasta: {bgs_dir.absolute()}")
print(f"Pasta existe: {bgs_dir.exists()}")

In [ ]:
# Listar todos os arquivos de imagem na pasta
if bgs_dir.exists():
    # Extensões de imagem suportadas
    image_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff']
    
    # Encontrar todos os arquivos de imagem
    image_files = []
    for file in bgs_dir.iterdir():
        if file.is_file() and file.suffix.lower() in image_extensions:
            image_files.append(file)
    
    # Ordenar por nome para ter uma sequência consistente
    image_files.sort()
    
    print(f"Encontrados {len(image_files)} arquivos de imagem:")
    for i, file in enumerate(image_files[:10]):  # Mostrar apenas os primeiros 10
        print(f"  {i+1}. {file.name}")
    
    if len(image_files) > 10:
        print(f"  ... e mais {len(image_files) - 10} arquivos")
else:
    print("Pasta 'bgs' não encontrada!")

In [ ]:
# Função para renomear arquivos com segurança
def rename_backgrounds(dry_run=True):
    if not bgs_dir.exists():
        print("Pasta 'bgs' não encontrada!")
        return
    
    # Encontrar arquivos de imagem
    image_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff']
    image_files = []
    for file in bgs_dir.iterdir():
        if file.is_file() and file.suffix.lower() in image_extensions:
            image_files.append(file)
    
    # Ordenar por nome
    image_files.sort()
    
    print(f"{'SIMULAÇÃO' if dry_run else 'RENOMEANDO'} {len(image_files)} arquivos:")
    print("-" * 60)
    
    renamed_count = 0
    
    for i, old_file in enumerate(image_files):
        # Novo nome sequencial com extensão .png
        new_name = f"bg{i+1}.png"
        new_path = bgs_dir / new_name
        
        # Verificar se já não é o nome correto
        if old_file.name == new_name:
            print(f"  {i+1:3d}. {old_file.name} → (já está correto)")
            continue
        
        # Verificar se o novo nome já existe
        if new_path.exists() and new_path != old_file:
            print(f"  {i+1:3d}. {old_file.name} → {new_name} (CONFLITO - arquivo já existe)")
            continue
        
        print(f"  {i+1:3d}. {old_file.name} → {new_name}")
        
        if not dry_run:
            try:
                # Se não for .png, converter para .png
                if old_file.suffix.lower() != '.png':
                    # Abrir e converter para PNG
                    with Image.open(old_file) as img:
                        # Converter para RGB se necessário (para PNG)
                        if img.mode in ('RGBA', 'LA', 'P'):
                            # Manter transparência se existir
                            img.save(new_path, 'PNG')
                        else:
                            # Converter para RGB
                            rgb_img = img.convert('RGB')
                            rgb_img.save(new_path, 'PNG')
                    
                    # Remover arquivo original
                    old_file.unlink()
                else:
                    # Apenas renomear
                    old_file.rename(new_path)
                
                renamed_count += 1
            except Exception as e:
                print(f"       ERRO: {str(e)}")
    
    print("-" * 60)
    if dry_run:
        print(f"SIMULAÇÃO concluída. {len([f for f in image_files if f.name != f'bg{image_files.index(f)+1}.png'])} arquivos seriam renomeados.")
        print("Execute com dry_run=False para renomear de fato.")
    else:
        print(f"Renomeação concluída. {renamed_count} arquivos renomeados com sucesso.")

# Executar simulação primeiro
rename_backgrounds(dry_run=True)

In [ ]:
# EXECUTE APENAS DEPOIS DE VERIFICAR A SIMULAÇÃO ACIMA
# Descomente a linha abaixo para renomear de fato

# rename_backgrounds(dry_run=False)

In [ ]:
# Verificar resultado final
if bgs_dir.exists():
    final_files = [f for f in bgs_dir.iterdir() if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff']]
    final_files.sort()
    
    print(f"Arquivos finais na pasta bgs ({len(final_files)} total):")
    for i, file in enumerate(final_files):
        expected_name = f"bg{i+1}.png"
        status = "✅" if file.name == expected_name else "❌"
        print(f"  {status} {file.name}")
        if i >= 20:  # Mostrar apenas os primeiros 20
            print(f"  ... e mais {len(final_files) - 20} arquivos")
            break
else:
    print("Pasta 'bgs' não encontrada!")